In [1]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

path = '/opt/ml/input/data/'
dat = pd.read_csv(path + 'FE/FE_total2.csv')

In [2]:
dat.drop(['day','year','KnowledgeTag', 'last_answerCode7','last_answerCode8','last_answerCode9','last_answerCode10'], axis = 1, inplace = True)

In [3]:
_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

In [4]:
'''
Train-Valid-Test
'''
# valid 제작하는 함수 새로 개편.
# 예전 valid 제작 함수는 1분 걸렸는데 0.6초만에 끗~
_train['train_valid'] = 0
_train.loc[_train.drop_duplicates(subset='userID', keep = 'last').index, 'train_valid'] = -1
_valid = _train[_train['train_valid'] == -1]
_train = _train[_train['train_valid'] == 0]

# 모델에 적용하기 전 기본적인 데이터 전처리 부분
## 라벨링, 필요없는 칼럼 제거
_train_value = _train['answerCode']
_train.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_valid_value = _valid['answerCode']
_valid.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_test.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

# CatBoost에 적용하기 위해선 문자열 데이터로 변환 필요.
# 만약 범주형이 아니고 연속형(정수, 소수) 성질을 살리고 싶은 변수는 여기 drop에 적어두면 됨.(solve_time 같이.)
cat_columns = list(_train.drop(['solve_time','month'],axis=1).columns)

for col in cat_columns:
    _train[col] = _train[col].astype('str')
    _valid[col] = _valid[col].astype('str')
    _test[col] = _test[col].astype('str')

In [5]:
'''
Train-Test
'''
# 모델에 적용하기 전 기본적인 데이터 전처리 부분
## 라벨링, 필요없는 칼럼 제거
_train_value = _train['answerCode']
_train.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_test.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'


# CatBoost에 적용하기 위해선 문자열 데이터로 변환 필요.
# 만약 범주형이 아니고 연속형(정수, 소수) 성질을 살리고 싶은 변수는 여기 drop에 적어두면 됨.(solve_time 같이.)
cat_columns = list(_train.drop(['solve_time','month'],axis=1).columns)

for col in cat_columns:
    _train[col] = _train[col].astype('str')
    _test[col] = _test[col].astype('str')

In [5]:
#!pip install catboost
# 가상환경에 패키지 설치 필요할듯
from catboost import CatBoostClassifier

# 하이퍼파라미터 수정할 여지가 많으니 추가나 수정하고 싶은 사람 수정해도 괜찮을듯.
# 일단 지난번에 사용했던 파라미터들 주석으로 들고 옴. 참고하면서 공부하면 좋을듯.

params_cat = {
    "task_type" : "GPU",
    "devices" : '0',
    "random_state": 42,
    'learning_rate': 0.01, 
    'bagging_temperature': 0.15696396388661144, 
    'n_estimators': 4000, 
    'max_depth': 15, 
    'random_strength': 13, 
    'l2_leaf_reg': 5.887526043950164e-06, 
    'min_child_samples': 9, 
    'max_bin': 297, 
    'od_type': 'Iter',
    "cat_features" : cat_columns
}

cat_model = CatBoostClassifier(**params_cat)

# 실제 적용한 모델은 learning_rate, iterations(=n_estimators, 아마도?)만 사용함.
#cat_model = CatBoostClassifier(learning_rate=0.01, random_state = 1998, iterations=4000, task_type="GPU", eval_metric = 'AUC',max_depth = 8)
cat_model.fit(_train, _train_value)

0:	learn: 0.6881988	total: 1.39s	remaining: 1h 32m 45s
1:	learn: 0.6833709	total: 2.67s	remaining: 1h 28m 57s
2:	learn: 0.6786399	total: 4s	remaining: 1h 28m 52s
3:	learn: 0.6740454	total: 5.34s	remaining: 1h 28m 52s
4:	learn: 0.6696288	total: 6.65s	remaining: 1h 28m 32s
5:	learn: 0.6654134	total: 8.03s	remaining: 1h 29m 3s
6:	learn: 0.6611620	total: 9.35s	remaining: 1h 28m 54s
7:	learn: 0.6570701	total: 10.7s	remaining: 1h 29m 3s
8:	learn: 0.6533665	total: 10.9s	remaining: 1h 20m 50s
9:	learn: 0.6493299	total: 12.2s	remaining: 1h 21m 23s
10:	learn: 0.6455012	total: 13.6s	remaining: 1h 21m 56s
11:	learn: 0.6425243	total: 13.7s	remaining: 1h 15m 39s
12:	learn: 0.6386840	total: 14.9s	remaining: 1h 16m 25s
13:	learn: 0.6350596	total: 15.8s	remaining: 1h 14m 46s
14:	learn: 0.6313282	total: 17.2s	remaining: 1h 15m 57s
15:	learn: 0.6284466	total: 17.3s	remaining: 1h 11m 44s
16:	learn: 0.6247715	total: 18.6s	remaining: 1h 12m 38s
17:	learn: 0.6214120	total: 20s	remaining: 1h 13m 34s
18:	learn

In [8]:
# valid score를 확인해봄.
# train 대비 valid score가 떨어짐. 과적합이 일어남. 마찬가지로 규제하는 하이퍼파라미터 필요.
from sklearn.metrics import accuracy_score, roc_auc_score

print('train score')
_train_pred = cat_model.predict_proba(_train)[:,1]
print(roc_auc_score(_train_value, _train_pred)) # auc
print(accuracy_score(_train_value, np.where(_train_pred >= 0.5, 1, 0))) # acc, 정확도

print('valid score')
_valid_pred = cat_model.predict_proba(_valid)[:,1]
print(roc_auc_score(_valid_value, _valid_pred)) # auc
print(accuracy_score(_valid_value, np.where(_valid_pred >= 0.5, 1, 0))) # acc, 정확도

train score
0.9209817822693678
0.8527618850051228


In [9]:
# submission 제출하기 위한 코드

_test_pred = cat_model.predict_proba(_test)[:,1]
_test['prediction'] = _test_pred
submission = _test['prediction'].reset_index(drop = True).reset_index()
submission.rename(columns = {'index':'id'}, inplace = True)
submission.to_csv('../output/cat1208.csv', index = False)